In [1]:
#from arcgis.gis import GIS
#from arcgis.mapping import WebMap
#from arcgis.features import FeatureLayer
#from IPython.display import display, HTML
#from ipywidgets import interact, interactive, fixed, interact_manual
#import ipywidgets as widgets

#gis = GIS(client_id = '6bs2nZZsgDYTM7Ut')
gis = GIS()
#print("Successfully logged in as: " + gis.properties.user.username)

NameError: name 'GIS' is not defined

In [57]:
#Creating list of disidx using temp map (temp map is arbitruary, canopy map contains disparity indexes too)
#list for use in drop down in main function
temp_map = gis.content.get('7f456e6543f1450bb4c1d672a79daaab')
temp_map_w = WebMap(temp_map)
layers = temp_map_w.layers
layer = layers[0]
attributes = layer['popupInfo']['fieldInfos']
attribute_list = []
for attribute in attributes:
    name = attribute['label']
    attribute_list.append(name)
start = attribute_list.index('DisIdxPoverty')
end = attribute_list.index('DisIdxBus')
disidx = attribute_list[start:end+1]


def disparity_index(disparity_index):
    #surface temperature map will be rendered using dispairty indexes
    cuse_layer = FeatureLayer('https://services1.arcgis.com/VM2sYETMYd1aOIXK/arcgis/rest/services/Surface_Temperature_Cuse_WFL1/FeatureServer')
    cuse_map = gis.map("Syracuse, NY")
    # Rendering
    disp_renderer = {"renderer": "autocast",
                 "type": "classBreaks",  
                 "field": disparity_index,
                 "minValue":-100}
    #colors in rgb, 4th value is transparency
    white = [255, 254, 230, 255]
    red1 = [255, 179, 179, 255]
    red2 = [255, 133, 133, 255]
    red3 = [255, 87, 87, 255]
    red4 = [255, 43, 43, 255]
    red5 = [255, 0, 0, 255]
    outline = {"color": [153,153,153,255], "width": 0.375,"type": "esriSLS", "style": "esriSLSSolid"}
    # Rendering class breaks
    disp_renderer["classBreakInfos"] = [
                                        
                                        {"symbol": {
                                            "color": white,
                                            "outline": outline,
                                            "type": "esriSFS",
                                            "style": "esriSFSSolid"},
                                        "classMaxValue": 0},
                                    
                                         {"symbol": {
                                            "color": red1,
                                            "outline": outline,
                                            "type": "esriSFS",
                                            "style": "esriSFSSolid"},
                                        "classMaxValue": 1},
                                        
                                        
                                        {"symbol": {
                                            "color": red2,
                                            "outline": outline,
                                            "type": "esriSFS",
                                            "style": "esriSFSSolid"}, 
                                         "classMaxValue": 2},
                                        
                                        {"symbol": {
                                            "color": red3,
                                            "outline": outline,
                                            "type": "esriSFS",
                                            "style": "esriSFSSolid"
                                        },
                                        "classMaxValue": 3},
                                      
                                        {"symbol": {
                                            "color": red4,
                                            "outline": outline,
                                            "type": "esriSFS",
                                            "style": "esriSFSSolid"
                                        },
                                        "classMaxValue": 6},
                        
                                        {"symbol": {
                                            "color": red5,
                                            "outline": outline,
                                            "type": "esriSFS",
                                            "style": "esriSFSSolid"
                                        },
                                        "classMaxValue": 69}]
    
    #add layer with rendering to blank map of syracuse
    cuse_map.add_layer(cuse_layer,
               { "type": "FeatureLayer",
                 "renderer": disp_renderer,
                 "field_name": disparity_index,
                  "minValue": -100})
    return cuse_map

#these functions return weird print message instead of map when called within main function
def temp_map():
    temp_map = gis.content.get('7f456e6543f1450bb4c1d672a79daaab')
    temp_map_w = WebMap(temp_map)
    return temp_map_w
def canopy_map():
    canopy_map = gis.content.get('c158a162af6b4130af5cc2f050b41dc6')
    canopy_map_w = WebMap(canopy_map)
    return canopy_map_w
display(HTML(f"<h2>What would you like to explore?"))

#gets the rank fields in a list for use in drop down in main function
webmap = gis.content.get('e679923ed6e14204b2426677044d110b')
p_layer = WebMap(webmap)
p_attributes = p_layer.layers
fields = p_attributes[0]['popupInfo']['fieldInfos']
field_list = []
for field in fields:
    name = field['fieldName']
    field_list.append(name)
start = field_list.index('Poverty_rank')
end = field_list.index('Temp_rank')
ranks = field_list[start:end+1]

#gets priority map and priority map features
priority = gis.content.get('eeb7fa098a544c4cb82903d394585483')
priority_layer = priority.layers[0]
query = priority_layer.query()
features = query.features

#gets user editable map
custom = gis.content.get('f9ae1ac3c476483f94e658e50bfe96e4')
c_layer = custom.layers[0]
c_query = c_layer.query()
c_features = c_query.features
priority_features = [f for f in c_features]
# if f.attributes['Priority_rank']==1 to above line?

#appends features
def feature_scores(feature):
    feature_values = []
    for f in features:
        feature_values.append(f.attributes[feature])
    return feature_values

def get_Priorties(poverty_weight, latino_weight, black_weight, senior_weight, 
                  child_weight, walk_weight, bus_weight, canopy_weight, temp_weight):
    # returns statement to user if weights do not add up to 100
    total_weight = poverty_weight + latino_weight + black_weight + senior_weight + child_weight + walk_weight + bus_weight + canopy_weight + temp_weight
    if total_weight != 100:
        return f"Weights must equal 100 in total, weights currently equal {total_weight}"
    # conversions from percents
    poverty_weight = poverty_weight * .01
    latino_weight = latino_weight * .01
    black_weight = black_weight * .01
    senior_weight = senior_weight * .01
    child_weight = child_weight * .01
    walk_weight = walk_weight * .01
    bus_weight = bus_weight * .01
    canopy_weight = canopy_weight * .01
    temp_weight = temp_weight * .01
    
    # scores
    poverty_scores = feature_scores('Poverty_rank')
    latino_scores = feature_scores('Latino_rank')
    black_scores = feature_scores('Black_rank')
    senior_scores = feature_scores('Senior_rank')
    child_scores = feature_scores('Children_rank')
    walk_scores = feature_scores('Walk_rank')
    bus_scores = feature_scores('Bus_rank')
    canopy_scores = (feature_scores('Canopy_rank'))
    temp_scores = feature_scores('Temp_rank')
    priority_list = []

    #appends the scores and inverses canopy
    for f in range(len(features)):
        priority_list.append(poverty_scores[f]*poverty_weight+latino_scores[f]*latino_weight+black_scores[f]*black_weight +
                            senior_scores[f]*senior_weight + child_scores[f]*child_weight + walk_scores[f]*walk_weight +
                            bus_scores[f]*bus_weight + (1-canopy_scores[f])*canopy_weight + temp_scores[f]*temp_weight)
    #makes a list of each census tract with calculated priority rank
    c_edit = c_features
    i = 0
    for f in c_edit:
        f.attributes['Priority_rank'] = priority_list[i]
        i+=1
    j = 0
    #updates feature layer with calculated prioritiy ranks
    for f in c_edit:
        update_result = c_layer.edit_features(updates=[c_edit[j]])
        j+=1
    custom_map = gis.map("Syracuse, NY")
    custom_map.add_layer(c_layer, { "renderer":"UnclassedColorsRenderer", "field_name":"Priority_rank" })
    custom_map
    return custom_map

def main(topic):
    if topic == 'Surface temperature':
        temp_map = gis.content.get('7f456e6543f1450bb4c1d672a79daaab')
        mymap = WebMap(temp_map)
        mymap
    elif topic == 'Canopy cover':
        canopy_map = gis.content.get('c158a162af6b4130af5cc2f050b41dc6')
        mymap = WebMap(canopy_map)
        mymap
    elif topic == 'Disparity indexes':
        interact(disparity_index, disparity_index=disidx)
    else:
        interact(get_Priorties, poverty_weight=widgets.IntSlider(min=0, max=100, step=1, value=15), 
                 latino_weight=widgets.IntSlider(min=0, max=100, step=1, value=15),
                black_weight=widgets.IntSlider(min=0, max=100, step=1, value=15),
                senior_weight=widgets.IntSlider(min=0, max=100, step=1, value=5),
                child_weight=widgets.IntSlider(min=0, max=100, step=1, value=5),
                walk_weight=widgets.IntSlider(min=0, max=100, step=1, value=10),
                bus_weight=widgets.IntSlider(min=0, max=100, step=1, value=10),
                canopy_weight=widgets.IntSlider(min=0, max=100, step=1, value=15),
                temp_weight=widgets.IntSlider(min=0, max=100, step=1, value=10))
    return mymap

In [58]:
interact(main, topic=['Surface temperature', 'Canopy cover', 'Disparity indexes', 'User concern index'])

interactive(children=(Dropdown(description='topic', options=('Surface temperature', 'Canopy cover', 'Disparity…

<function __main__.main(topic)>

<FeatureLayer url:"https://services1.arcgis.com/VM2sYETMYd1aOIXK/arcgis/rest/services/Surface_Temperature_Cuse_WFL1/FeatureServer">

ERROR: Could not open requirements file: [Errno 2] No such file or directory: 'requirements.txt'
